In [2]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

# common_voice["train"] = load_dataset("mozilla-foundation/common_voice_17_0", "yue", split="train+validation", trust_remote_code=True)
# common_voice["test"] = load_dataset("mozilla-foundation/common_voice_17_0", "yue", split="test", trust_remote_code=True)
common_voice = load_dataset("JasonLeeYiMan/unique_location_name_in_hongkong", split="train")
common_voice = common_voice.train_test_split(test_size=0.2, shuffle=True, seed=42)
print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['audio', 'text'],
        num_rows: 374
    })
    test: Dataset({
        features: ['audio', 'text'],
        num_rows: 94
    })
})


In [3]:
from transformers import WhisperFeatureExtractor
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-large-v3-turbo")

In [4]:
from transformers import WhisperTokenizer
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-large-v3-turbo", language="yue", task="transcribe")

In [5]:
from transformers import WhisperProcessor
processor = WhisperProcessor.from_pretrained("openai/whisper-large-v3-turbo", language="yue", task="transcribe")

2025-08-14 03:42:07.344225: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-14 03:42:07.364828: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755142927.399943   60564 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755142927.408130   60564 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755142927.428861   60564 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [6]:
print(common_voice["train"][100])

{'audio': {'path': 'Sun_Hing_Garden_gd.wav', 'array': array([0., 0., 0., ..., 0., 0., 0.]), 'sampling_rate': 16000}, 'text': '大埔，新興花園'}


In [7]:
from datasets import Audio
common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

In [8]:
print(common_voice["train"][100])

{'audio': {'path': 'Sun_Hing_Garden_gd.wav', 'array': array([0., 0., 0., ..., 0., 0., 0.]), 'sampling_rate': 16000}, 'text': '大埔，新興花園'}


In [9]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]
    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    # encode target text to label ids
    batch["labels"] = tokenizer(batch["text"]).input_ids
    return batch

In [10]:
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=1)

Map:   0%|          | 0/374 [00:00<?, ? examples/s]

Map:   0%|          | 0/94 [00:00<?, ? examples/s]

In [11]:
from transformers import WhisperForConditionalGeneration
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v3-turbo")

In [12]:
model.generation_config.language = "yue"
model.generation_config.task = "transcribe"
model.generation_config.forced_decoder_ids = None

In [13]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # # split inputs and labels since they have to be of different lengths and need different padding methods
        # # first treat the audio inputs by simply returning torch tensors
        # input_features = [{"input_features": feature["input_features"]} for feature in features]
        # batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # # get the tokenized label sequences
        # label_features = [{"input_ids": feature["labels"]} for feature in features]

        # labels_batch = self.processor.tokenizer.pad(
        #     label_features,
        #     return_tensors="pt"
        # )

        # # replace padding with -100 to ignore loss correctly
        # labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # # if bos token is appended in previous tokenization step,
        # # cut bos token here as it's append later anyways
        # if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
        #     labels = labels[:, 1:]

        # batch["labels"] = labels
        # return batch

        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        # 🔄 CHANGE: Manual EOS padding implementation
        # Find the maximum length
        max_length = max(len(feature["input_ids"]) for feature in label_features)

        # Manually pad with EOS tokens and create attention masks
        padded_labels = []
        attention_masks = []

        for feature in label_features:
            input_ids = feature["input_ids"]
            padding_length = max_length - len(input_ids)

            # Pad with EOS tokens
            padded = input_ids + [self.processor.tokenizer.eos_token_id] * padding_length
            padded_labels.append(padded)

            # Create attention mask (1 for real tokens, 0 for padding)
            attention_mask = [1] * len(input_ids) + [0] * padding_length
            attention_masks.append(attention_mask)

        # Convert to tensors
        labels_tensor = torch.tensor(padded_labels, dtype=torch.long)
        attention_mask_tensor = torch.tensor(attention_masks, dtype=torch.long)

        # Create labels_batch-like structure for compatibility
        labels_batch = {
            "input_ids": labels_tensor,
            "attention_mask": attention_mask_tensor
        }

        # replace EOS padding tokens with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch["attention_mask"].ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch



In [14]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

In [17]:
import evaluate

metric = evaluate.load("wer")

In [18]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [15]:
from transformers import Seq2SeqTrainingArguments
from datetime import datetime

OUTPUT_MODEL_NAME = "whisper_large_v3_turbo_yue"
username = "thomaskywong"
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
repo_name = f"{username}/{OUTPUT_MODEL_NAME}_{timestamp}"

# training_args = Seq2SeqTrainingArguments(
#     output_dir="./whisper_large_v3_turbo_yue",  # change to a repo name of your choice
#     per_device_train_batch_size=2,
#     gradient_accumulation_steps=8,  # increase by 2x for every 2x decrease in batch size
#     learning_rate=1e-5,
#     warmup_steps=500,
#     max_steps=334,                 # originally 4000
#     gradient_checkpointing=False,
#     fp16=True,
#     eval_strategy="steps",
#     per_device_eval_batch_size=8,
#     predict_with_generate=True,
#     generation_max_length=225,
#     save_steps=333,                 # originally 1000
#     eval_steps=333,                 # originally 1000
#     logging_steps=25,
#     report_to=["tensorboard"],
#     load_best_model_at_end=True,
#     metric_for_best_model="wer",
#     greater_is_better=False,
#     push_to_hub=True,
#     hub_model_id=repo_name,  # ADD THIS LINE - use the repo_name you created earlier
# )

training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/whisper-large-v3-turbo-yue",
    per_device_train_batch_size=2,    # decrease by factor of 2 in case of out-of-memory (VRAM)
    gradient_accumulation_steps=8,    # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    num_train_epochs=5,               # Changed from max_steps to epochs
    gradient_checkpointing=False,
    fp16=True,
    eval_strategy="epoch",            # Evaluate after each epoch
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_strategy="epoch",            # Save after each epoch
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
    hub_model_id=repo_name,
    hub_strategy="end",               # Push final model only
)

In [19]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    processing_class=processor,
)

HfHubHTTPError: (Request ID: Root=1-689d5d5f-372118c45f0cd6aa60c5bfbb;31bfa7c5-9dc1-439c-a7a0-8732ce29bc9e)

403 Forbidden: You don't have the rights to create a model under the namespace "thomaskywong".
Cannot access content at: https://huggingface.co/api/repos/create.
Make sure your token has the correct permissions.

In [ ]:
trainer.train()

In [ ]:
kwargs = {
    "dataset_tags": "mozilla-foundation/common_voice_17_0",
    "dataset": "Common Voice 17.0",  # a 'pretty' name for the training dataset
    "dataset_args": "config: yue, split: test",
    "language": "yue",
    "model_name": "Whisper Large V3 Turbo Yue - Thomas Wong",  # a 'pretty' name for our model
    "finetuned_from": "openai/whisper-large-v3-turbo",
    "tasks": "automatic-speech-recognition",
}

In [ ]:
trainer.push_to_hub(**kwargs)